# 9.2 Implement our own system assembling

In this tutorial we

* write an integrator for $\int_T \nabla u \nabla v$

* put together element matrices to the global system matrix

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square

from ngsolve.fem import CompilePythonModule
from pathlib import Path

txt = Path('myassemblemodule.cpp').read_text() 
m = CompilePythonModule(txt, init_function_name='mymodule', add_header=False)
dir (m)

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=3, dirichlet=".*")
u, v = fes.TnT()

### use our own integrators for element matrix calculation:

In [ ]:
a = BilinearForm(fes)
a += m.MyLaplace(CF(1))
a.Assemble()

f = LinearForm(fes)
f += m.MySource(1)
f.Assemble();

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);

### use our own matrix assembling function:

In [ ]:
with TaskManager():
    mymatrix = m.MyAssembleMatrix(fes, m.MyLaplace(CF(1)))
gfu.vec.data = mymatrix.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);
# print ("my matrix = ", mymat)